# Ground Truth and Step Count Merge

## Setup

In [1]:
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.filterwarnings("ignore")

## Load and Check Data Integrity

**Coding Ground Truth**

In [3]:
# Import coding ground truth
ground_truth = pd.read_csv("ACT24_groundtruth_secbysec.csv", dtype={"work_type": "string"}) #dtype since missing values were stored as NaN, should be as pd.na

In [4]:
# Display first 5 rows for inspection
ground_truth.head()

,id,observation,date,date_time,activity_type,broad_activity_type,work_type,posture,sedentary_not,walking_not,activity_intensity
0,102,1,2019-07-24,2019-07-24 08:20:19,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light
1,102,1,2019-07-24,2019-07-24 08:20:20,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light
2,102,1,2019-07-24,2019-07-24 08:20:21,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light
3,102,1,2019-07-24,2019-07-24 08:20:22,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light
4,102,1,2019-07-24,2019-07-24 08:20:23,WRK- general,work_education,SP- Education and Health Services,stand,not_sedentary,not_walking,light


In [5]:
# Check for duplicate timestamps
check = ground_truth[["id", "observation", "date_time"]].value_counts()
check[check > 1]

Series([], dtype: int64)

**Steps Ground Truth**

In [6]:
# Import seconds ground truth
seconds = pd.read_csv("seconds_ground_truth.csv")

In [7]:
# Display first 5 rows for inspection
seconds.sort_values(["ID", "Session", "Date", "Time"]).head()

,ID,Session,Relative Time,Date,Time,Quality,Step
114078,102,1,0:00:01,7/24/2019,08:20:20,Codable,0
114079,102,1,0:00:02,7/24/2019,08:20:21,Codable,0
114080,102,1,0:00:03,7/24/2019,08:20:22,Codable,0
114081,102,1,0:00:04,7/24/2019,08:20:23,Codable,0
114082,102,1,0:00:05,7/24/2019,08:20:24,Codable,1


In [8]:
# Check for duplicate timestamps
check = seconds[["ID", "Session", "Date", "Time"]].value_counts()
# Duplicate timestamps were found, check for specific rows and step counts
check[check > 1].reset_index().drop(0, axis=1).merge(right = seconds, on=["ID", "Session", "Date", "Time"])

,ID,Session,Date,Time,Relative Time,Quality,Step
0,117,1,8/20/2019,05:36:54,2:19:08,Codable,0
1,117,1,8/20/2019,05:36:54,2:19:08,Codable,0
2,127,1,9/5/2019,04:24:03,0:16:07,Codable,0
3,127,1,9/5/2019,04:24:03,0:16:07,Codable,0
4,127,1,9/5/2019,04:24:10,0:16:14,Codable,0
...,...,...,...,...,...,...,...
152065,133,1,10/10/2019,04:02:10,1:30:14,Codable,1
152066,133,1,10/10/2019,03:37:28,1:05:32,Codable,0
152067,133,1,10/10/2019,03:37:28,1:05:32,Codable,0
152068,127,1,9/5/2019,04:30:03,0:22:07,Codable,0


## Ground Truth Data Cleaning

In [9]:
def quality_agg(srs):
    if len(srs) > 1:
        return "Ambiguous"
    return srs.iloc[0]

In [ ]:
seconds_2 = seconds.copy()
# Rename columns
seconds_2.rename(columns={"ID": "id", "Session": "observation"}, inplace=True)
# Remove duplicates
# Logic: Get max step count for each timestamp
seconds_2 = seconds_2.groupby(["id", "observation", "Date", "Time", "Relative Time"]).agg({'Quality': quality_agg, 'Step': 'max'}).reset_index()
# Create datetime column
seconds_2["date_time"] = pd.to_datetime(seconds_2['Date'] + ' ' + seconds_2['Time'])
seconds_2

In [ ]:
# Check for duplicates again
check = seconds_2[["id", "observation", "date_time"]].value_counts()
check[check > 1]

In [ ]:
ground_truth_2 = ground_truth.copy()
# Create datetime column
ground_truth_2["date_time"] = pd.to_datetime(ground_truth["date_time"])
ground_truth_2

## Check Start and End Times of Both Files

In [ ]:
# Function to get start and end info for each id and session
def start_end(GT, id, session): 
    subset = GT[(GT["id"] == id) & (GT["observation"] == session)].reset_index(drop=True)
    start = min(subset["date_time"])
    end = max(subset["date_time"])
    for i, row in subset.iterrows():
        if i != 0:
            if row["date_time"] < start:
                start = row["date_time"]
            elif row["date_time"] > end:
                end = row["date_time"]
    return [start, end]
    #print(f"ID = {id}, Session = {session}, Start = {start}, End = {end}")
        

In [ ]:
# Function to apply start_end to entire dataframe (ie. each id and session)
def start_time_dataframe(df):
    id_ses = df.groupby(["id", "observation"]).size().reset_index().rename(columns={0: "count"})
    times = {"ID": [], 
                   "Session": [],
                   "Start": [],
                   "End": []}
    for i, row in id_ses.iterrows():
        times["ID"].append(row["id"])
        times["Session"].append(row["observation"])
        ls = start_end(df, row["id"], row["observation"])
        times["Start"].append(ls[0])
        times["End"].append(ls[1])
    return pd.DataFrame(times)

In [ ]:
# Get start and end times from both coding and step ground truths
GT_times = start_time_dataframe(ground_truth_2)
seconds_times = start_time_dataframe(seconds_2)

In [ ]:
start_end_info = pd.merge(GT_times, seconds_times, on=["ID", "Session"], how="outer", suffixes=("_GT", '_secondsFile'))
# Get differences between seconds/coding ground truth, in seconds
start_end_info["StartDiffSecs(GT-seconds)"] = (start_end_info["Start_GT"] - start_end_info["Start_secondsFile"]).dt.total_seconds()
start_end_info["EndDiffSecs(GT-seconds)"] = (start_end_info["End_GT"] - start_end_info["End_secondsFile"]).dt.total_seconds()
start_end_info

**Add Issue Flags in Columns**

In [ ]:
# Function that flags id/observation pairs if the times between step and coding ground truth are off by around 12 hrs
def flag_military(row):
    if row > 40000:
        return True
    else:
        return False

In [ ]:
start_end_info_2 = start_end_info.copy()
start_end_info_2["MilitaryTimeFlag"] = start_end_info_2["StartDiffSecs(GT-seconds)"].apply(flag_military)
start_end_info_2

**Issues:** 
* Negative start second differences implies that the ground truth start times are OUTSIDE the seconds file session times
* Positive end second differences implies that the ground truth end times are OUTSIDE the seconds file session times

**This implies that we have tagged times for this (ID, Session) pair in our ground truth file that have no information about steps taken :(**

**1\. Ground Truth File contains military time timestamps but seconds file is in AM/PM**
* MilitaryTimeFlag = True means this row has this problem
* **NOTE**: Fixed.

**2\. (ID, Session) pairs missing in seconds file (9 total)**
* Probably just need to throw these out since we do not have step info for these :(

## Fix AM/PM Disparity

In [ ]:
start_end_info_3 = start_end_info_2.copy()
# Fix start AM/PM
start_end_info_3['Start_secondsFile'] = start_end_info_3['Start_secondsFile'] + pd.to_timedelta("12:00:00") * start_end_info_3['MilitaryTimeFlag']
# Fix end AM/PM
start_end_info_3['End_secondsFile'] = start_end_info_3['End_secondsFile'] + pd.to_timedelta("12:00:00") * start_end_info_3['MilitaryTimeFlag']
# Recalculate differences
start_end_info_3["StartDiffSecs(GT-seconds)"] = (start_end_info_3["Start_GT"] - start_end_info_3["Start_secondsFile"]).dt.total_seconds()
start_end_info_3["EndDiffSecs(GT-seconds)"] = (start_end_info_3["End_GT"] - start_end_info_3["End_secondsFile"]).dt.total_seconds()
# Recode military time flag
start_end_info_3['MilitaryTimeFlag'] = start_end_info_3["StartDiffSecs(GT-seconds)"].apply(flag_military)
# We need to determine which rows in the second-by-second data fall within the intersection of the groundtruth time intervals, since they don't match up
# Find the greatest start time
start_end_info_3['max_start'] = start_end_info_3[['Start_GT', 'Start_secondsFile']].max(axis=1)
# Find the smallest end time
start_end_info_3['min_end'] = start_end_info_3[['End_GT', 'End_secondsFile']].min(axis=1)
start_end_info_3

In [ ]:
# Check to make sure there are no more AM/PM disparities
start_end_info_3[start_end_info_3['MilitaryTimeFlag']]

In [ ]:
# Edit steps ground truth to fix AM/PM disparities
seconds_3 = seconds_2.merge(right = start_end_info_2[["ID", "Session", "MilitaryTimeFlag", "StartDiffSecs(GT-seconds)"]], how = 'left',
              left_on = ["id", "observation"], right_on = ["ID", "Session"])
seconds_3["date_time"] = seconds_3["date_time"] + pd.to_timedelta("12:00:00") * seconds_3["MilitaryTimeFlag"]
seconds_3 = seconds_3[["id", "observation", "date_time", "Quality", "Step"]]
seconds_3

In [ ]:
# Check for duplicate times
check = seconds_3[["id", "observation", "date_time"]].value_counts()
check[check>1]

## Merge Ground Truth Datasets

In [ ]:
# Outer merge coding with steps ground truth
merged = pd.merge(left = ground_truth_2, right = seconds_3.drop("Quality", axis=1), how="outer", on=["id", "observation", "date_time"])
# Add flag for times that are inside the max start and min end interval
merged = merged.merge(right = start_end_info_3[["ID", "Session", "max_start", "min_end"]], how = 'left',
                            left_on = ["id", "observation"], right_on = ["ID", "Session"])
merged["inside_flag"] = (merged["date_time"] >= merged["max_start"]) & (merged["date_time"] <= merged["min_end"])
merged = merged.drop(["ID", "Session", "max_start", "min_end"], axis=1)
merged

In [ ]:
# Check to make sure that there is only one row per second
check = merged[["id", "observation", "date_time"]].value_counts()
check[check > 1]

## Check Valid Rows (Intersection of Steps, Coding Tables)

At this point, we know that the rows are invalid if:

1. The ID and Session/observation are missing from the seconds ground truth entirely, not partially
2. OR They fall outside of where the time intervals of the two ground truth files intersect.

We need to know if any more rows are invalid. The only rows that could be invalid at this point are rows that:

1. Have a valid ID/Session
2. AND they fall within the time interval
3. AND they either don't have steps or coding ground truth

We need to check if any such rows exist.

In [ ]:
# List of id/session that were found in the seconds file and in ground truth file
id_session_notna = start_end_info_3[["ID", "Session"]][~start_end_info_3["Start_secondsFile"].isna()]
id_session_notna

In [ ]:
# Find all valid rows - rows that should have data from both coding and step ground truth
# They should be in the valid id/sessions
merged_valid = id_session_notna.merge(right = merged, left_on = ["ID", "Session"], right_on = ["id", "observation"]).drop(["ID", "Session"], axis=1)
# They should also fall within the time interval between the max start and min end of both coding/step ground truth
merged_valid = merged_valid[merged_valid["inside_flag"] == True]
merged_valid

In [ ]:
# Check if there are any invalid rows remaining - there should be none
merged_valid[(merged_valid["walking_not"].isna()) | (merged_valid["Step"].isna())]

Based on our analysis, there are no invalid rows that:

1. Have a valid ID/Session
2. AND they fall within the time interval
3. AND they either don't have steps or coding ground truth

## Analyze Time Interval Disparity

Here we show the rows that have a valid ID/Session but fall outside the time interval intersection of the two ground truth tables. They should match the sum of the difference in seconds of the start/end that we calculated in the start_end_info tables.

In [ ]:
# Find the total number of seconds/rows that fall outside the max start/min end interval
total_outside = start_end_info_3['StartDiffSecs(GT-seconds)'].abs().sum() + start_end_info_3['EndDiffSecs(GT-seconds)'].abs().sum()
total_outside

In [ ]:
merged_outside = merged[merged["inside_flag"] == False]
merged_outside

The numbers match up. We have the correct number of rows that fall outside the interval. Because they fall outside of the interval intersection, they will only have steps or coding ground truth, not both.

We can further separate the two into rows that are in only coding and rows that are only in step count (that fall outside the common time interval intersection).

In [ ]:
# Only in step count
sc_only = merged_outside[merged_outside["activity_type"].isna()]
sc_only

In [ ]:
# Only in coding
gt_only = merged_outside[merged_outside["Step"].isna()]
gt_only

Some further analysis on the rows that are only in the coding ground truth:

In [ ]:
gt_only_summary = gt_only['activity_type'].value_counts()
gt_only_summary = gt_only_summary.reset_index()
gt_only_summary = gt_only_summary.rename(columns = {"activity_type": "category"})
gt_only_summary['column'] = 'activity_type'

for col in ['broad_activity_type', 'work_type', 'posture', 'sedentary_not', 'walking_not']:
    new_summary = gt_only[col].value_counts()
    new_summary = new_summary.reset_index()
    new_summary = new_summary.rename(columns = {col: "category"})
    new_summary['column'] = col
    gt_only_summary = pd.concat([gt_only_summary, new_summary], axis=0)
    
gt_only_summary.groupby(["column", "category"]).sum()

**IMPORTANT NOTE**: Observed, most rows are non-walking, sedentary, sitting. We may re-add these rows to the dataset if we can confirm from Dr. Keadle that we can assume a step count of 0 for these rows.

Some further analysis on the rows only in step count:

In [ ]:
sc_only['Step'].value_counts()

## Analyze Missing IDs/Sessions from Steps Ground Truth

These rows correspond to coding ground truth ID/Sessions that are missing from the steps ground truth entirely.

In [ ]:
# List of id/session that were found in ground truth file but NOT seconds file
id_session_na = start_end_info_3[["ID", "Session"]][start_end_info_3["Start_secondsFile"].isna()]
id_session_na

In [ ]:
merged_invalid_idsession = id_session_na.merge(right = merged, left_on = ["ID", "Session"], right_on = ["id", "observation"]).drop(["ID", "Session"], axis=1)
merged_invalid_idsession

## New Time Intervals

In [ ]:
final_start_end = start_time_dataframe(merged_valid)
final_start_end

Compare to log:

In [ ]:
log = pd.read_csv("do_log_final.csv")
log["Start"] = pd.to_datetime(log["start_month"].astype(str) + "/" +
                              log["start_day"].astype(str) + "/" +
                              log["start_year"].astype(str) + "/" + ' ' +
                              log["start_time"])
log["End"] = pd.to_datetime(log["start_month"].astype(str) + "/" +
                              log["start_day"].astype(str) + "/" +
                              log["start_year"].astype(str) + "/" + ' ' +
                              log["stop_time"])
log = log[['id', 'do', 'Start', 'End']].rename(columns = {"do": "Session", "id": "ID"})
log

In [ ]:
log_compare = final_start_end.merge(log, on=["ID", "Session"], how="left", suffixes=("_final", "_log"))
log_compare["Start_diff"] = (log_compare["Start_final"] - log_compare["Start_log"]).dt.total_seconds()
log_compare["End_diff"] = (log_compare["End_final"] - log_compare["End_log"]).dt.total_seconds()
log_compare

## Final Table

**Final Ground Truth with Invalid Rows Dropped**

In [ ]:
final_gt = merged_valid.drop(["inside_flag"], axis=1).rename(columns={"Step": "step"})
final_gt['step'] = final_gt['step'].astype(int)
final_gt

In [ ]:
final_gt.to_csv("merged_groundtruth_secbysec.csv", index=False)